```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Riverbend Election Example

An illustrative social simulation with 5 players which simulates the day of mayoral elections in an imaginary town caller Riverbend. First two players, Alice and Bob, are running for the mayor. The third player, Charlie, is trying to ruin Alices' reputation with disinformation. The last two players have no specific agenda, apart from voting in the election.

## Init and import

In [ ]:
# @title Imports

import collections
import concurrent.futures
import datetime
import random

from google.colab import widgets
from IPython import display

from concordia import components
from concordia.agents import basic_agent
from concordia.agents import components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import embedder_st5
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.environment import components as gm_components
from concordia.environment import game_master
from concordia.language_model import sax_model
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from concordia.examples.village.components import elections
from concordia.examples.village.metrics import elections as elections_metrics

In [ ]:
# @title Setup sentence encoder
embedder = embedder_st5.EmbedderST5()

In [ ]:
# @title SAX Language Model

# Add path to your SAX server here:
SAX_PATH = '' # @param {type:"string"}
DEFAULT_MAX_TOKENS = 300  # @param {type: 'integer'}
DEFAULT_TIMEOUT_SECONDS = 60  # @param {type: 'number'}

model = sax_model.SAXLanguageModel(
    SAX_PATH,
    deterministic_multiple_choice=True,
)

## Configuring the generic knowledge of players and GM.

In [ ]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'There is a hamlet named Riverbend.',
    'Riverbend is an idyllic rural town.',
    'The river Solripple runs through the village of Riverbend.',
    'The Solripple is a mighty river.',
    'Riverbend has a temperate climate.',
    'Riverbend has a main street.',
    'There is a guitar store on Main street Riverbend.',
    'There is a grocery store on Main street Riverbend.',
    'There is a school on Main street Riverbend.',
    'There is a library on Main street Riverbend.',
    'Riverbend has only one pub.',
    'There is a pub on Main street Riverbend called The Sundrop Saloon.',
    'Town hall meetings often take place at The Sundrop Saloon.',
    'Riverbend does not have a park',
    'The main crop grown on the farms near Riverbend is alfalfa.',
    'Farms near Riverbend depend on water from the Solripple river.',
    (
        'The local newspaper recently reported that someone has been dumping '
        + 'dangerous industrial chemicals in the Solripple river.'
    ),
    'All named characters are citizens. ',
    # 'All citizens are automatically candidates in all elections. ',
    'There is no need to register in advance to be on the ballot.',
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [ ]:
#@title Make the clock
SETUP_TIME = datetime.datetime(hour=8, year=2024, month=9, day=1)

START_TIME = datetime.datetime(hour=9, year=2024, month=10, day=1)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[datetime.timedelta(hours=1), datetime.timedelta(seconds=10)])


## Functions to build the players

In [ ]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [ ]:
def build_a_citizen(agent_config,
                    player_names: list[str],
                    measurements: measurements_lib.Measurements | None = None):

  mem = formative_memory_factory.make_memories(agent_config)

  # Build the player.

  time = generic_components.report_function.ReportFunction(
    name='Current time',
    function=clock.current_time_interval_str,
  )
  somatic_state = components.somatic_state.SomaticState(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
  )
  identity = components.identity.SimIdentity(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
  )
  goal_component = generic_components.constant.ConstantComponent(state=agent_config.goal)
  plan = components.plan.SimPlan(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      components=[identity],
      goal=goal_component,
      verbose=False,
  )
  current_obs = components.observation.Observation(
      agent_name=agent_config.name,
      memory=mem)
  summary_obs = components.observation.ObservationSummary(
      model=model,
      agent_name=agent_config.name,
      components=[identity],
  )
  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )
  morality_metric = common_sense_morality.CommonSenseMoralityMetric(
      model=model,
      player_name=agent_config.name,
      clock=clock,
      name='Morality',
      verbose=False,
      measurements=measurements,
      channel='common_sense_morality',
  )
  agent = basic_agent.BasicAgent(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=True,
      components=[
          identity,
          plan,
          somatic_state,
          summary_obs,
          current_obs,
          time,
          goal_metric,
          morality_metric,
      ],
  )
  reputation_metric = opinion_of_others.OpinionOfOthersMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Opinion',
      verbose=False,
      measurements=measurements,
      channel='opinion_of_others',
      question='What is {opining_player}\'s opinion of {of_player}?',
  )
  agent.add_component(reputation_metric)

  return agent

In [ ]:
def context_from_role(player_name, role, victim='Alice'):
  context = f'{shared_context}\n{player_name} grew up in Riverbend.'
  # Divide players into two classes, half are biased toward the status quo and
  # the other half want change.
  if 'conservative' in role:
    context = (
        f'{context} {player_name} feels strongly that Riverbend is '
        + 'already a great place to live and does not need to change.'
    )
  if 'progressive' in role:
    context = (
        f'{context} {player_name} feels strongly that Riverbend and '
        + 'its local government are in dire need of reform.'
    )
  # The first two players are mayoral candidates.
  if 'candidate' in role:
    context = (
        f'{context} Because of this, {player_name} plans to run for '
        + 'mayor of Riverbend.'
    )
  if 'active_voter' in role:
    context = (
        f'{context} {player_name} does not plan to run for mayor of Riverbend,'
        ' but would definitely vote in the election.'
    )

  # The third player has been hired to ruin the first player's reputation.
  if 'corrupt' in role:
    context = (
        f'{context}\n'
        + f"{player_name} has been hired to ruin {victim}'s "
        + 'reputation.\n'
        + f'{player_name} was hired by an anonymous email so '
        + f"they do not know who hired them to ruin {victim}'s "
        + 'reputation or what their motivation may be.\n'
        + f'{player_name} was given fake compromising material on'
        f' {victim}.\n{player_name} was offered a substantial sum of'
        ' money to spread compromising materials '
        + f"to ruin {victim}'s reputation."
    )

  return context


## Configure and build the players

In [ ]:
victim = 'Alice'

def make_random_big_five()->str:
  return str({
      'extraversion': random.randint(1, 10),
      'neuroticism': random.randint(1, 10),
      'openness': random.randint(1, 10),
      'conscientiousness': random.randint(1, 10),
      'agreeableness': random.randint(1, 10),
  })

player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Win the election and become the mayor of Riverbend',
        context=context_from_role('Alice', {'candidate', 'conservative'}),
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Bob',
        gender='male',
        goal='Win the election and become the mayor of Riverbend.',
        context=context_from_role('Bob', {'candidate', 'progressive'}),
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Charlie',
        gender='male',
        goal=f"Ruin {victim}'s reputation",
        context=context_from_role('Charlie', {'corrupt'}, victim),
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal='Have a good day and vote in the election.',
        context=context_from_role(
            'Dorothy', {'active_voter', 'progressive'}
        ),
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
    formative_memories.AgentConfig(
        name='Ellen',
        gender='female',
        goal=(
            'Have a good day and vote in the election.'
        ),
        context=context_from_role('Ellen', {'active_voter', 'conservative'}),
        traits = make_random_big_five(),
        formative_ages = sorted(random.sample(range(5, 40), 5)),
    ),
]

In [ ]:
NUM_PLAYERS = 5

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {
    player_config.name: player_config.goal for player_config in player_configs}
players = []
measurements = measurements_lib.Measurements()

player_names = [player.name for player in player_configs][:NUM_PLAYERS]
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_a_citizen,
                        player_configs[:NUM_PLAYERS],
                        # All players get the same `player_names`.
                        [player_names] * NUM_PLAYERS,
                        # All players get the same `measurements` object.
                        [measurements] * NUM_PLAYERS):
    players.append(agent)

## Build GM

In [ ]:
game_master_instructions = (
    'This is a social science experiment. It is structured as a '
    'tabletop roleplaying game (like dungeons and dragons). You are the '
    'game master. You will describe the current situation to the '
    'participants in the experiment and then on the basis of what you '
    'tell them they will suggest actions for the character they control. '
    'Aside from you, each other participant controls just one character. '
    'You are the game master so you may control any non-player '
    'character. You will track the state of the world and keep it '
    'consistent as time passes in the simulation and the participants '
    'take actions and change things in their world. Remember that this '
    'is a serious social science experiment. It is not just a game. It '
    'need not be fun for the participants. Always use third-person '
    'limited perspective, even when speaking directly to the participants.'
)

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now)

In [ ]:
for player in players:
  game_master_memory.add(f'{player.name} is at their private home.')

In [ ]:
# @title Create components and externalities
player_names = [player.name for player in players]

instructions_construct = components.constant.ConstantComponent(game_master_instructions, 'Instructions')
facts_on_village = components.constant.ConstantComponent(' '.join(shared_memories), 'General knowledge of Riverbend')
player_status = gm_components.player_status.PlayerStatus(clock.now, model, game_master_memory, player_names)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

election_externality = elections.Elections(
    model=model,
    clock_now=clock.now,
    memory=game_master_memory,
    voters=players,
    candidates=['Alice', 'Bob'],
    verbose=True,
    measurements=measurements,
)

mem_factory = blank_memories.MemoryFactory(
    model,
    embedder,
    importance_model_gm.importance,
    clock_now=clock.now,
)

convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=mem_factory,
    components=[player_status],
    cap_nonplayer_characters=2,
    game_master_instructions=game_master_instructions,
    shared_context=shared_context,
    verbose=True,
)


direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)


TIME_POLLS_OPEN = datetime.datetime(hour=14, year=2024, month=10, day=1)
TIME_POLLS_CLOSE = datetime.datetime(hour=20, year=2024, month=10, day=1)
schedule = {
    'start': gm_components.schedule.EventData(
        time=START_TIME,
        description=None),
    'election': gm_components.schedule.EventData(
        time=datetime.datetime(hour=13, year=2024, month=10, day=1),
        description=(
            'The town of Riverbend is now holding an election to determine ' +
            'who will become the mayor. ' +
            f'Polls will open at {TIME_POLLS_OPEN}.')),
    'election_polls_open': gm_components.schedule.EventData(
        time=TIME_POLLS_OPEN,
        description=(
            'The election is happening now. Polls are open. Everyone may ' +
            'go to a polling place and cast their vote. ' +
            f'Polls will close at {TIME_POLLS_CLOSE}.'),
        trigger=election_externality.open_polls),
    'election_polls_close': gm_components.schedule.EventData(
        time=TIME_POLLS_CLOSE,
        description=(
            'The election is over. Polls are now closed. The results will ' +
            'now be tallied and a winner declared.'),
        trigger=election_externality.declare_winner)
}

schedule_construct = gm_components.schedule.Schedule(
    clock_now=clock.now, schedule=schedule)


In [ ]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        instructions_construct,
        facts_on_village,
        player_status,
        schedule_construct,
        election_externality,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [ ]:
clock.set(START_TIME)

In [ ]:
for player in players:
  player.observe(
      f'{player.name} is at home, they have just woken up. Mayoral elections '
      f'are going to be held today. Polls will open at {TIME_POLLS_OPEN} and '
      f'close at {TIME_POLLS_CLOSE}.'
  )
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for player in players:
    pool.submit(player.update)

In [ ]:
# @title Expect about 2-3 minutes per step.

episode_length = 12  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()


## Summary and analysis of the episode

In [ ]:
# @title Metrics plotting

colab_import.reload_module(plotting)

group_by = collections.defaultdict(lambda: 'player')
group_by['opinion_of_others'] = 'of_player'

tb = widgets.TabBar([channel for channel in measurements.available_channels()])
for channel in measurements.available_channels():
  with tb.output_to(channel):
    plotting.plot_line_measurement_channel(measurements, channel,
                                           group_by=group_by[channel],
                                           xaxis='time_str')


In [ ]:
# @title Summarize the entire story.
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = '\n'.join(all_gm_memories)
print('len(detailed_story): ', len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f'Sequence of events:\n{detailed_story}'+
    '\nNarratively summarize the above temporally ordered ' +
    'sequence of events. Write it as a news report. Summary:\n',
    max_characters=8000, max_tokens=8000, terminators=())
print(episode_summary)

In [ ]:
# @title Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(player._memory.retrieve_recent(k=1000,
                                                            add_time=True))
  summary = ''
  summary = model.sample_text(
      f'Sequence of events that happened to {name}:\n{detailed_story}'
      '\nWrite a short story that summarises these events.\n'
      ,
      max_characters=8000, max_tokens=8000, terminators=())

  all_player_mem = player._memory.retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')


#Build and display HTML log of the experiment

In [ ]:
history_sources = [
    env,
    direct_effect_externality,
    convo_externality,
    election_externality,
]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources]
histories_names = [history.name() for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    histories_html + [gm_mem_html] + player_logs,
    histories_names + ['GM'] + player_log_names,
    summary=episode_summary,
    title='Riverbend elections experiment',
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

#Interact with a specific player

In [ ]:
sim_to_interact = 'Alice'  # @param ['Alice', 'Bob','Charlie', 'Dorothy', 'Ellen'] {type:"string"}
user_identity = 'a close friend'  # @param {type:"string"}
interaction_premise = f'{sim_to_interact} is talking to {user_identity}\n'  # @param {type:"string"}

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterence_from_user = 'Did you win the election?'  # @param {type:"string"}

interrogation += f'{user_identity}: {utterence_from_user}'
player_says = selected_player.say(interrogation)
interrogation += f'\n{sim_to_interact}: {player_says}\n'
print(interrogation)